In [42]:
!pip install -q langchain
!pip install -q transfomers sentencepiece accelerate bitsandbytes
!pip install -q sentence_transformers
!pip install -q python-dotenv
!pip install langchain-google-genai
!pip install google-colab
!pip install langchain langchain-community
!pip install -q unstructured
!pip install nltk
!pip install -q wikipedia
!pip install -q pypdf
!pip install -q tiktoken
!pip install chromadb
!pip install -q faiss-cpu

ERROR: Could not find a version that satisfies the requirement transfomers (from versions: none)
ERROR: No matching distribution found for transfomers
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.7 MB/s eta 0:00:00


In [40]:
from dotenv import dotenv_values
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage
from langchain_community.chat_models import ChatGooglePalm
import os
import pandas as pd
from langchain.document_loaders import DataFrameLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import WikipediaLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import OnlinePDFLoader
from langchain.document_loaders import DirectoryLoader
from transformers import AutoTokenizer
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS

env_values = dotenv_values("./app.env")

google_api_key = env_values['GOOGLE_API_KEY']

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7,
    google_api_key=google_api_key
)

In [30]:
query1="Nikola Tesla"
query2="Thomas Edison"

docs1=WikipediaLoader(query=query1,load_max_docs=1,doc_content_chars_max=20000).load()
docs2=WikipediaLoader(query=query2,load_max_docs=1,doc_content_chars_max=20000).load()

text_document1=docs1[0].page_content
text_documnet2=docs2[0].page_content
documents=[text_document1,text_documnet2]
metadatas=[{"source":query1},{"source":query2}]

In [31]:
documents[0]

'Nikola Tesla (; Serbian Cyrillic: Никола Тесла [nǐkola têsla]; 10 July 1856 – 7 January 1943) was a Serbian-American engineer, futurist, and inventor. He is known for his contributions to the design of the modern alternating current (AC) electricity supply system.\nBorn and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without receiving a degree. He then gained practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884 he immigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in New York to develop a range of electrical and mechanical devices. His AC induction motor and related polyphase AC patents, licensed by Westinghouse Electric in 1888, earned him a c

In [32]:
from langchain.text_splitter import NLTKTextSplitter
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [33]:
text_splitter = NLTKTextSplitter(chunk_size=300, chunk_overlap=50)
tokens_chunks = text_splitter.create_documents(documents, metadatas=metadatas)
len(tokens_chunks)

177

In [34]:
tokens_chunks[5]

Document(metadata={'source': 'Nikola Tesla'}, page_content='Attempting to develop inventions he could patent and market, Tesla conducted a range of experiments with mechanical oscillators/generators, electrical discharge tubes, and early X-ray imaging.\n\nHe also built a wirelessly controlled boat, one of the first ever exhibited.')

In [24]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"

embedding_llm = SentenceTransformerEmbeddings(model_name=model_name)

<ipython-input-24-2050278359>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_llm = SentenceTransformerEmbeddings(model_name=model_name)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.w

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [35]:
docs_text=[chunk.page_content for chunk in tokens_chunks]

docs_embeddings = embedding_llm.embed_documents(docs_text)

In [36]:
query_text = "Can you list a number of Nikola Tesla's inventions?"
query_embedding = embedding_llm.embed_query(query_text)

In [37]:
docs_embeddings[0]

[-0.13145188987255096,
 0.10117759555578232,
 -0.04324796423316002,
 0.06852191686630249,
 -0.08107732236385345,
 -0.00606581661850214,
 0.0248715840280056,
 0.06821466237306595,
 -0.05090930312871933,
 -0.006575395818799734,
 0.0533742718398571,
 0.012287193909287453,
 0.060044754296541214,
 0.04953049495816231,
 -0.10668852180242538,
 0.021365247666835785,
 -0.06624055653810501,
 0.03421413525938988,
 -0.017229652032256126,
 -0.005048387683928013,
 -0.10046523064374924,
 0.014396660961210728,
 -0.0027750413864851,
 -0.09684772044420242,
 0.08496716618537903,
 0.048667844384908676,
 -0.021389955654740334,
 -0.002305585192516446,
 0.026487451046705246,
 0.012782902456820011,
 -0.023822564631700516,
 -0.09221016615629196,
 -0.05854899436235428,
 0.049133095890283585,
 -0.011171706020832062,
 0.009351610206067562,
 -0.0176628939807415,
 0.05845152586698532,
 0.10239960253238678,
 0.015938036143779755,
 -0.012714705429971218,
 -0.051085758954286575,
 -0.032335780560970306,
 0.036308381706

In [43]:
vector_db=FAISS.from_documents(tokens_chunks,embedding_llm)

In [51]:
query_text = "In which year did Tesla's fathre die?"
similar_docs=vector_db.similarity_search(query_text,k=2)
similar_docs

[Document(id='1acb5311-e0d9-4dad-b795-691105ae76f4', metadata={'source': 'Nikola Tesla'}, page_content="He died in New York City in January 1943.\n\nTesla's work fell into relative obscurity following his death, until 1960, when the General Conference on Weights and Measures named the International System of Units (SI) measurement of magnetic flux density the tesla in his honor."),
 Document(id='69e14b76-4d52-4a98-9ea1-8241c64ca857', metadata={'source': 'Nikola Tesla'}, page_content='After Wardenclyffe, Tesla experimented with a series of inventions in the 1910s and 1920s with varying degrees of success.\n\nHaving spent most of his money, Tesla lived in a series of New York hotels, leaving behind unpaid bills.\n\nHe died in New York City in January 1943.')]

In [52]:
save_to_dir = "/content/wiki_chroma_db"

docs_ids = list( range( len(tokens_chunks) ) )
docs_ids = [ str(d) for d in docs_ids ]

vector_db = Chroma.from_documents(
                                tokens_chunks,
                                embedding_llm,
                                persist_directory=save_to_dir,
                                ids=docs_ids
                            )

In [62]:
query_text = "In which year did Tesla's fathre die?"

similar_docs = vector_db.similarity_search(
                                      query_text,
                                      k=5,
                                      filter={'source': 'Nikola Tesla'}
                                    )
vector_db.persist()

In [56]:
load_from_dir = "/content/wiki_chroma_db"

loaded_vector_db = Chroma(
    persist_directory=load_from_dir,
    embedding_function=embedding_llm
)

<ipython-input-56-4140417104>:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  loaded_vector_db = Chroma(


In [61]:
similar_docs = loaded_vector_db.similarity_search(
                                      query_text,
                                      k=5,
                                      filter={'source': 'Nikola Tesla'}
                                    )
print(similar_docs)

[Document(metadata={'source': 'Nikola Tesla'}, page_content="He died in New York City in January 1943.\n\nTesla's work fell into relative obscurity following his death, until 1960, when the General Conference on Weights and Measures named the International System of Units (SI) measurement of magnetic flux density the tesla in his honor."), Document(metadata={'source': 'Nikola Tesla'}, page_content='After Wardenclyffe, Tesla experimented with a series of inventions in the 1910s and 1920s with varying degrees of success.\n\nHaving spent most of his money, Tesla lived in a series of New York hotels, leaving behind unpaid bills.\n\nHe died in New York City in January 1943.'), Document(metadata={'source': 'Nikola Tesla'}, page_content="Tesla's father died the next month, on 17 April 1879, at the age of 60 after an unspecified illness.\n\nDuring the rest of the year Tesla taught a large class of students in his old school in Gospić."), Document(metadata={'source': 'Nikola Tesla'}, page_conte